# Ethereum Patricia Merkle Tries Explained


In [37]:
from ethereum import utils
import rlp
import ethereum.trie as trie
import ethereum.db as db


A merkle patricia trie is a combination of two types of tries: a merkle trie and a special type of radix trie, a patricia trie. This combination makes it easy to add/retrieve stored values and also efficient to provide proof that a specific value exists.

We will be adding four different keys in our merkle patricia trie, to make it easier for us, the keys will be hex numbers that look very similar

In [38]:
first_key = b'\x01\x02\x03'
second_key = b'\x01\x02\x04'
third_key = b'\x01\x02'
fourth_key = b'\x01\x02\x03\x78'


We define a few functions to be able to print the contents of the trie

In [39]:
def print_node_information(t):
    print('----------------------------------')
    print('root hash:', utils.encode_hex(t.root_hash))
    k, v = t.root_node
    print ('root node:', [k, v])
    print ('root node in hex:', [utils.encode_hex(k), utils.encode_hex(v)])
    print ('hp encoded key, in hex:', utils.encode_hex(k))
    print('----------------------------------')
    print('\n')

def print_node(t):
    _, v = t.root_node
    node = t._decode_to_node(v)
    for i, val in enumerate(node):
        toPrint = val
        if val == b'':
            toPrint = ''
            print("(", i,")", toPrint, end="  ", flush=True)
        elif isinstance(val, (list, tuple)):
            print("( %d ) %s  %s" % (i, val[0], (val[1])), end="  ", flush=True)
        else:
            toPrint = (val)
            print("(", i,")", toPrint, end="  ", flush=True)

    print('')


We initialize the trie, and validate that the root hash of the trie is the same as the keccack256 hash of the rlp encoded empty string ""

In [40]:

# Create a Trie with an ephemeral database
t = trie.Trie(db.EphemDB())
# The trie will be initialized with a blank_node which is the SHA# rlp encoded of ""
empty_root = t.root_hash
print(utils.encode_hex(t.root_hash))
# both of these are equivalent:
print(utils.encode_hex(utils.sha3rlp(b'')))


56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421
56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421


Let's add a key value pair to the trie. For this example, we will add the key "first" with value "node"

In [41]:
# Let's update our trie by adding a node
t.update(rlp.encode("first"), rlp.encode("node"))
# Here we notice the root has is changed
print(utils.encode_hex(t.root_hash))

# Let's get the key and value at the root node
k, v = t.root_node
# We print it
print('root node:', [k, v])
print('hp encoded key, in hex', utils.encode_hex(k))


4a02922e58e52ef6f66bc1ea3ca4f699558ec488973cfd0ee8a3767a9523418f
root node: [b' \x85first', b'\x84node']
hp encoded key, in hex 20856669727374


Notice that the root node hash has changed. Also, we notice that the key and value are directly stored on the root node, no need to traverse the trie to get "first"

Why the [b' \x85first', b'\x84node']? This is part of the RLP encoding. 

The HP specification is rather simple. A nibble is appended to the key that encodes both the terminator status and parity. The lowest significant bit in the nibble encodes parity, while the next lowest encodes terminator status. If the key was in fact even, then we add another nibble, of value 0, to maintain overall evenness (so we can properly represent in bytes).



Here we get root node: [b' \x85first', b'\x84node'] with a hex value of 208466697273 for "first"
The first 2 bits (20) are important. 2 indicates that it is a terminator node, and the 0 was added to keep the
key even.

Now same thing with a hex key
Let's delete our first node

Let's delete the element, leaving us with an empty trie to play around with 

In [42]:
t.delete(rlp.encode("first"))
print(utils.encode_hex(t.root_hash))
# ensure that we have the same root hash
print(t.root_hash == empty_root)

56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421
True


In [43]:

# Let's update our trie by adding a node
t.update(first_key, rlp.encode("node"))
# Here we notice the root has is changed
print(utils.encode_hex(t.root_hash))

# # Let's get the key and value at the root node
k, v = t.root_node
# # We print it
print('root node:', [k, v])
print('hp encoded key, in hex', utils.encode_hex(k))

5c81fd48e14e3a07b6f06cba66a0d53f5604a2c40a75563cc120994c8e0f0c27
root node: [b' \x01\x02\x03', b'\x84node']
hp encoded key, in hex 20010203


In [44]:
# What happens if you put another value at the same key? We replace it
t.update(first_key, rlp.encode("anotherNode"))
print(t.get(first_key))



b'\x8banotherNode'


In [47]:
# Now let's add to the trie. Let's add a third value
t.update(second_key, rlp.encode("newentry"))

k, v = t.root_node
print_node_information(t)


----------------------------------
root hash: 3ae6fd5f86a90a008708beb18de52faebb7a7aff40325c22b7899501d980baea
root node: [b'\x10\x10 ', b'\xdcJ~G\xd4En\xdd\x0b\xed^\xd3\xd8\x9b\xd6\xdb&]\rH\xa9[\xf2\xd1s:\xfdy\x98\x1a\xe7\x90']
root node in hex: ['101020', 'dc4a7e47d4456edd0bed5ed3d89bd6db265d0d48a95bf2d1733afd79981ae790']
hp encoded key, in hex: 101020
----------------------------------





Ok now it gets more complex, we've added a new node with a different extension at the end. Extension node!
So we see that the value is now a hash, this is a hash of where we can find the node in the DB

In [48]:
print(t._get_node_type(t.root_node) == trie.NODE_TYPE_EXTENSION)

True


Let's get the node using the last value we got:

In [49]:
new_node = t._decode_to_node(v)
print_node(t)


( 0 )   ( 1 )   ( 2 )   ( 3 ) b' '  b'\x8banotherNode'  ( 4 ) b' '  b'\x88newentry'  ( 5 )   ( 6 )   ( 7 )   ( 8 )   ( 9 )   ( 10 )   ( 11 )   ( 12 )   ( 13 )   ( 14 )   ( 15 )   ( 16 )   


This is our last node, so it should be of type branch

In [50]:
# This is our last node, so it should be of type branch
print (t._get_node_type(new_node) == trie.NODE_TYPE_BRANCH)
# What we have here is a branch node, a list with 17 entries.
print(len(new_node))
# Their positions is important, our nodes are at position 3 and 4 in the array, which is the difference between the two keys
print(new_node[3], new_node[4])
# The keys were:
print(utils.encode_hex(first_key), utils.encode_hex(second_key))


True
17
[b' ', b'\x8banotherNode'] [b' ', b'\x88newentry']
010203 010204


Now let's mess things up, let's add something that has a shorter key

In [51]:
t.update(third_key, rlp.encode('anothernewnode'))
k, v = t.root_node
print_node_information(t)

new_node = t._decode_to_node(v)
print_node(t)


----------------------------------
root hash: 884db17039f7443833f478c5c2f64153b3b32794b1267f5f118499631f755d78
root node: [b'\x00\x01\x02', b"\xad+'E\x0c\x8cu\xe3\xe7J\x19\x06\xea\xaaG\x0e\xc6\xb1\x19\xc8\x14\xe3\xf1UV\xe2\xda\x8f\x9b7\xae9"]
root node in hex: ['000102', 'ad2b27450c8c75e3e74a1906eaaa470ec6b119c814e3f15556e2da8f9b37ae39']
hp encoded key, in hex: 000102
----------------------------------


( 0 ) b'\xdcJ~G\xd4En\xdd\x0b\xed^\xd3\xd8\x9b\xd6\xdb&]\rH\xa9[\xf2\xd1s:\xfdy\x98\x1a\xe7\x90'  ( 1 )   ( 2 )   ( 3 )   ( 4 )   ( 5 )   ( 6 )   ( 7 )   ( 8 )   ( 9 )   ( 10 )   ( 11 )   ( 12 )   ( 13 )   ( 14 )   ( 15 )   ( 16 ) b'\x8eanothernewnode'  


ok so here we see that "anothernode" has been saved at the 17th position, thus indicating that there is no more further to go. Now at position 0 (because of 010102, 0 is the next location) is where we have our node hash, so we need to go and get it

In [52]:
new_node_hash = new_node[0]
lower_node = t._decode_to_node(new_node_hash)
print_node(t)
# and we get our old node back


( 0 ) b'\xdcJ~G\xd4En\xdd\x0b\xed^\xd3\xd8\x9b\xd6\xdb&]\rH\xa9[\xf2\xd1s:\xfdy\x98\x1a\xe7\x90'  ( 1 )   ( 2 )   ( 3 )   ( 4 )   ( 5 )   ( 6 )   ( 7 )   ( 8 )   ( 9 )   ( 10 )   ( 11 )   ( 12 )   ( 13 )   ( 14 )   ( 15 )   ( 16 ) b'\x8eanothernewnode'  


Now let's add 1 last thing, now a longer key

In [56]:
t.update(fourth_key, rlp.encode('lastnode'))

We print it's information

In [55]:
k, v = t.root_node
print_node_information(t)
new_node = t._decode_to_node(v)
print_node(t)

ValueError: too many values to unpack (expected 2)

Now we have a new node at position 0 and 4 with the information that we want

In [53]:


mid_node = t._decode_to_node(new_node[0])
print_node(t)
last_node = t._decode_to_node(mid_node[3])
print_node(t)

----------------------------------
root hash: 2277b96cab8210feb20686f36c3ce78b1b33c2b56aa71a990df57f506af3b738
root node: [b'\x00\x01\x02', b'\xf4\x82r\x02vb\xd1=6mG\xdc\x13\x1a\xb5[\xcc\xd9RK\xa0)\xac$\xf6u\xcc\xaaq]\xaa0']
root node in hex: ['000102', 'f48272027662d13d366d47dc131ab55bccd9524ba029ac24f675ccaa715daa30']
hp encoded key, in hex: 000102
----------------------------------


( 0 ) b'~\x80\xf6\xec\xfc\xa1\xee\x12\x90[\xb3\xe8\xb8\xa1-?$W\xd6\xee\xc2Xv!M\xfb\x12\xf1\x87_\x84\xdb'  ( 1 )   ( 2 )   ( 3 )   ( 4 )   ( 5 )   ( 6 )   ( 7 )   ( 8 )   ( 9 )   ( 10 )   ( 11 )   ( 12 )   ( 13 )   ( 14 )   ( 15 )   ( 16 ) b'\x8eanothernewnode'  
( 0 ) b'~\x80\xf6\xec\xfc\xa1\xee\x12\x90[\xb3\xe8\xb8\xa1-?$W\xd6\xee\xc2Xv!M\xfb\x12\xf1\x87_\x84\xdb'  ( 1 )   ( 2 )   ( 3 )   ( 4 )   ( 5 )   ( 6 )   ( 7 )   ( 8 )   ( 9 )   ( 10 )   ( 11 )   ( 12 )   ( 13 )   ( 14 )   ( 15 )   ( 16 ) b'\x8eanothernewnode'  
( 0 ) b'~\x80\xf6\xec\xfc\xa1\xee\x12\x90[\xb3\xe8\xb8\xa1-?$W\xd6\xee\xc2Xv!M\xfb\x

Now how the state patricia merkle trie in ethereum works is that the keys are the accounts/contract addresses, so let's add one now with the value "code"

In [59]:
address = (utils.coerce_addr_to_hex(utils.remove_0x_head("0x864Be2775d392787D5fa37ee1DB45FE0b1B3D1FC")))
print(rlp.encode(address))

#rlp endode the address -- the trie wil be significantly longer than expected
t.update(rlp.encode(address), rlp.encode('code'))
# Root nodes becomes an extension node!
print(t.root_node)


address_location = t.root_node[10]
address_node = t._decode_to_node(address_location)
print(address_node)

print(t._get_node_type(address_node) == trie.NODE_TYPE_LEAF)

b'\xa8864Be2775d392787D5fa37ee1DB45FE0b1B3D1FC'
[b'\xc3\xb8\xad\xe1\x87\xfcV@\xee\xdem\xbee\xb8\x82\xdb\xf8\xe0\xc5\x17\xfe\xd4\xc1@\x82\xcd\xdfzC\x0c\xe1\x18', b'', b'', b'', b'', b'', b'', b'', b'', b'', b"9\xec\x93r|\x92\xb9\xbd1\xf6K@5q\xf3D=<\x1b\xe8dQ9\xd3\xac\x03jY,&'t", b'', b'', b'', b'', b'', b'']
[b'8864Be2775d392787D5fa37ee1DB45FE0b1B3D1FC', b'\x84code']
True
